### 1. Criação GloVe Embeddings

- Carregar os dados de cada site
    - Suno
    - Infomoney
    - MoneyTimes
- Realizar o preprocessamento
- Concatenar em arquivo .txt
- Realizar o procedimento descrito na documentação do GloVe

In [1]:
import re
import json
import numpy as np
import pandas as pd
import seaborn as sns
import string
import matplotlib.pyplot as plt

In [2]:
with open('../crawlers/suno/data/results-full-suno-2020.json', encoding='utf8') as json_file:
    data = json.load(json_file)

df_suno = pd.DataFrame(data)

In [3]:
with open('../crawlers/money-times/data/results-full-moneytimes-2020.json', encoding='utf8') as json_file:
    data = json.load(json_file)

df_moneytimes = pd.DataFrame(data)

In [4]:
with open('../crawlers/infomoney/data/results-full-infomoney-2020.json', encoding='utf8') as json_file:
    data = json.load(json_file)

df_infomoney = pd.DataFrame(data)

In [5]:
def remove_emojis(sentence):

    "Remoção de Emojis nas mensagens de texto."

    # Padrões dos Emojis
    emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)

    return emoji_pattern.sub(r'', sentence)

def remove_valores(sentence):
    new_sentece = ''
    
    for token in sentence.split():
        if token.isdigit():
            token = '<NUM>'
        new_sentece += ' {}'.format(token)
        
    return new_sentece

In [6]:
# 1. Aplicar preprocessamento nos títulos e textos completos

for df in [df_suno, df_infomoney, df_moneytimes]:

    # Substituir símbolos importantes
    df['title'] = df['title'].map(lambda s: s.replace('-feira', ''))
    df['title'] = df['title'].map(lambda s: s.replace('+', 'mais '))
    df['title'] = df['title'].map(lambda s: s.replace('-', 'menos '))
    df['title'] = df['title'].map(lambda s: s.replace('%', ' por cento'))
    df['title'] = df['title'].map(lambda s: s.replace('R$', ''))
    df['title'] = df['title'].map(lambda s: s.replace('U$', ''))
    df['title'] = df['title'].map(lambda s: s.replace('US$', ''))
    df['title'] = df['title'].map(lambda s: s.replace('S&P 500', 'spx'))

    df['full_text'] = df['full_text'].map(lambda s: s.replace('-feira', ''))
    df['full_text'] = df['full_text'].map(lambda s: s.replace('+', 'mais '))
    df['full_text'] = df['full_text'].map(lambda s: s.replace('-', 'menos '))
    df['full_text'] = df['full_text'].map(lambda s: s.replace('%', ' por cento'))
    df['full_text'] = df['full_text'].map(lambda s: s.replace('R$', ''))
    df['full_text'] = df['full_text'].map(lambda s: s.replace('U$', ''))
    df['full_text'] = df['full_text'].map(lambda s: s.replace('US$', ''))
    df['full_text'] = df['full_text'].map(lambda s: s.replace('S&P 500', 'spx'))

    # Transformar em String e Letras Minúsculas nas Mensagens
    df['title'] = df['title'].map(lambda s: str(s).lower())
    df['full_text'] = df['full_text'].map(lambda s: str(s).lower())

    # Remover Pontuações
    df['title'] = df['title'].map(lambda s: s.translate(str.maketrans('', '', string.punctuation)))
    df['full_text'] = df['full_text'].map(lambda s: s.translate(str.maketrans('', '', string.punctuation)))

    # Remover Emojis     
    df['title'] = df['title'].map(lambda s: remove_emojis(s))
    df['full_text'] = df['full_text'].map(lambda s: remove_emojis(s))

    # Quebras de Linha desnecessárias
    df['title'] = df['title'].map(lambda s: s.replace('\n', ' '))
    df['full_text'] = df['full_text'].map(lambda s: s.replace('\n', ' '))

    # Remover aspas duplas
    df['title'] = df['title'].map(lambda s: s.replace('\"', ''))
    df['title'] = df['title'].map(lambda s: s.replace('“', ''))
    df['title'] = df['title'].map(lambda s: s.replace('”', ''))

    df['full_text'] = df['full_text'].map(lambda s: s.replace('\"', ''))
    df['full_text'] = df['full_text'].map(lambda s: s.replace('“', ''))
    df['full_text'] = df['full_text'].map(lambda s: s.replace('”', ''))

    # Remover valores
    df['title'] = df['title'].map(lambda s: remove_valores(s))
    df['full_text'] = df['full_text'].map(lambda s: remove_valores(s))

    # Espaços desnecessários
    df['title'] = df['title'].map(lambda s: s.strip())
    df['full_text'] = df['full_text'].map(lambda s: s.strip())
    
    # Salvar os textos completos em arquivo .txt
    df['title'].to_csv('full_text.txt', index=False, mode='a')
    df['full_text'].to_csv('full_text.txt', index=False, mode='a')

#### Realizar procedimento da documentação do Glove...

---